In [ ]:
from numpy import array
u = array([1.1, .01]) 
type(u)

In [ ]:
dt_step      = 1.0 
dt_track     = 0.1
sim_duration = 20
step         = int(dt_step / dt_track)
ellipse_step = 2 * step
track_step   = int(sim_duration / dt_track)

In [ ]:
track_step

In [ ]:
breakpoint()  # 设置断点


In [ ]:
def func(i):
    x = i * 2
    y = i + 3
    return x, y  # 返回两个值作为元组
t = 6
T = 2 * t
# 调用函数，并将返回的两个值分别赋给 x 和 y
x, y = func(t)

print(f"x = {x}, y = {y}")


In [ ]:
import sympy as sp

# 定义符号变量
x_x, x_y, theta, v, a = sp.symbols('x_x x_y theta v a')

# 定义状态转移方程
f = sp.Matrix([
    x_x + v * sp.cos(theta),  # 预测下一个 x
    x_y + v * sp.sin(theta),  # 预测下一个 y
    theta + a                # 预测下一个方向
])

# 初始化数值
state = sp.Matrix([2.0, 3.0, 0.5])  # 当前状态 [x, y, theta]
control = sp.Matrix([1.1, 0.1])     # 控制输入 [v, a]

# 创建符号到数值的映射
subs = {
    x_x: state[0],
    x_y: state[1],
    theta: state[2],
    v: control[0],
    a: control[1]
}

# 将符号替换为数值
numerical_result = f.subs(subs)
print(numerical_result)


In [ ]:
f

In [ ]:
import numpy as np
import matplotlib.pylab as plt
import matplotlib.gridspec as gridspec
import pandas as pd
from bagpy import bagreader

# import data from recording at Klitten FVZ
results_file = "./Klitten/loc_results_2023-11-01-12-53-52_0.bag"
sensor_file = "./Klitten/recordedData_sensors_locpc_2023-11-01-12-53-52_0.bag"

result_reader = bagreader(results_file)
sensor_reader = bagreader(sensor_file)


# Thius function extracts the sensordata into CSV-files

map_path = result_reader.message_by_topic(topic="/odometry/filtered/map")
odom_path = result_reader.message_by_topic(topic="/odometry/filtered/odom")
gps_odometry_path  = result_reader.message_by_topic(topic="/odometry/gps")
imu_oem = sensor_reader.message_by_topic(topic="/localization_pc/can0/decoded/imu")
speed_oem = sensor_reader.message_by_topic(topic="/localization_pc/can0/decoded/speed")
steeringAngle_oem = sensor_reader.message_by_topic(topic="/localization_pc/can0/decoded/steeringAngle")
gnss_oem = sensor_reader.message_by_topic(topic="/localization_pc/can0/decoded/gnss")
imu_anavs = sensor_reader.message_by_topic(topic="/localization_pc/sensor/anavs/imu")
navsat_anavs = sensor_reader.message_by_topic(topic="/localization_pc/sensor/anavs/navsat")


# You can then use Pandas to read the files.
df_map_path = pd.read_csv(map_path)  
df_odom_path = pd.read_csv(odom_path)  
df_gps_path = pd.read_csv(gps_odometry_path)
df_speed = pd.read_csv(speed_oem)
df_imu = pd.read_csv(imu_oem)

# 我写的
df_imu_anavs = pd.read_csv(imu_anavs)
df_navsat_anvas = pd.read_csv(navsat_anavs)

In [1]:
import numpy as np
import matplotlib.pylab as plt
import matplotlib.gridspec as gridspec
import pandas as pd
from bagpy import bagreader

# import data from recording at Klitten FVZ
results_file = "./Klitten/loc_results_2023-11-01-12-53-52_0.bag"
sensor_file = "./Klitten/recordedData_sensors_locpc_2023-11-01-12-53-52_0.bag"

result_reader = bagreader(results_file)
sensor_reader = bagreader(sensor_file)


# Thius function extracts the sensordata into CSV-files

map_path = result_reader.message_by_topic(topic="/odometry/filtered/map")
#odom_path = result_reader.message_by_topic(topic="/odometry/filtered/odom")
#gps_odometry_path  = result_reader.message_by_topic(topic="/odometry/gps")
imu_oem = sensor_reader.message_by_topic(topic="/localization_pc/can0/decoded/imu")
#speed_oem = sensor_reader.message_by_topic(topic="/localization_pc/can0/decoded/speed")
#steeringAngle_oem = sensor_reader.message_by_topic(topic="/localization_pc/can0/decoded/steeringAngle")
#gnss_oem = sensor_reader.message_by_topic(topic="/localization_pc/can0/decoded/gnss")
#imu_anavs = sensor_reader.message_by_topic(topic="/localization_pc/sensor/anavs/imu")
#navsat_anavs = sensor_reader.message_by_topic(topic="/localization_pc/sensor/anavs/navsat")


# You can then use Pandas to read the files.
df_map_path = pd.read_csv(map_path)  
#df_odom_path = pd.read_csv(odom_path)  
# df_gps_path = pd.read_csv(gps_odometry_path)
#df_speed = pd.read_csv(speed_oem)
df_imu = pd.read_csv(imu_oem)

# 我写的
# df_imu_anavs = pd.read_csv(imu_anavs)
# df_navsat_anvas = pd.read_csv(navsat_anavs)

[INFO]  Data folder ./Klitten/loc_results_2023-11-01-12-53-52_0 already exists. Not creating.
[INFO]  Data folder ./Klitten/recordedData_sensors_locpc_2023-11-01-12-53-52_0 already exists. Not creating.


In [2]:
df_map_path['header.stamp.secs'] = df_map_path['header.stamp.secs'] - df_map_path['header.stamp.secs'].iloc[0]
df_imu['header.stamp.secs'] = df_imu['header.stamp.secs'] - df_imu['header.stamp.secs'].iloc[0]

In [3]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [4]:
# 计算新列
df_imu['secs.nsecs'] = df_imu['header.stamp.secs'] + df_imu['header.stamp.nsecs'] * 1e-9
df_imu['secs.nsecs'] = df_imu['secs.nsecs'].round(9)
# 调整列的位置，将 'secs.nsecs' 放到 'header.stamp.nsecs' 后面
columns = df_imu.columns.tolist()
index = columns.index('header.stamp.nsecs') + 1  # 获取目标列的索引并加1
columns.insert(index, columns.pop(columns.index('secs.nsecs')))  # 移动 secs.nsecs 列

# 重新排列列顺序
df_imu = df_imu[columns]

In [5]:
df_imu.iloc[955:962]

,Time,header.seq,header.stamp.secs,header.stamp.nsecs,secs.nsecs,header.frame_id,orientation.x,orientation.y,orientation.z,orientation.w,orientation_covariance_0,orientation_covariance_1,orientation_covariance_2,orientation_covariance_3,orientation_covariance_4,orientation_covariance_5,orientation_covariance_6,orientation_covariance_7,orientation_covariance_8,angular_velocity.x,angular_velocity.y,angular_velocity.z,angular_velocity_covariance_0,angular_velocity_covariance_1,angular_velocity_covariance_2,angular_velocity_covariance_3,angular_velocity_covariance_4,angular_velocity_covariance_5,angular_velocity_covariance_6,angular_velocity_covariance_7,angular_velocity_covariance_8,linear_acceleration.x,linear_acceleration.y,linear_acceleration.z,linear_acceleration_covariance_0,linear_acceleration_covariance_1,linear_acceleration_covariance_2,linear_acceleration_covariance_3,linear_acceleration_covariance_4,linear_acceleration_covariance_5,linear_acceleration_covariance_6,linear_acceleration_covariance_7,linear_acceleration_covariance_8
955,1.698840e+09,12177,19,944182001,19.944182,vehicle_link,0.0,0.0,0.0,1.0,-1.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,-1.0,0.0,0.0,-0.000524,0.01,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.000893,-0.0625,-0.04,0.0,0.000282,0.0,0.0,0.0,0.000003,0.0,0.0,0.0,0.01
956,1.698840e+09,12178,19,964895960,19.964896,vehicle_link,0.0,0.0,0.0,1.0,-1.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,-1.0,0.0,0.0,-0.000175,0.01,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.000893,-0.0625,-0.04,0.0,0.000282,0.0,0.0,0.0,0.000003,0.0,0.0,0.0,0.01
957,1.698840e+09,12179,19,994744518,19.994745,vehicle_link,0.0,0.0,0.0,1.0,-1.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,-1.0,0.0,0.0,-0.000524,0.01,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.000893,-0.0625,-0.04,0.0,0.000282,0.0,0.0,0.0,0.000003,0.0,0.0,0.0,0.01
958,1.698840e+09,12180,20,4796224,20.004796,vehicle_link,0.0,0.0,0.0,1.0,-1.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,-1.0,0.0,0.0,-0.000698,0.01,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.000893,-0.0625,-0.04,0.0,0.000282,0.0,0.0,0.0,0.000003,0.0,0.0,0.0,0.01
959,1.698840e+09,12181,20,34710262,20.034710,vehicle_link,0.0,0.0,0.0,1.0,-1.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,-1.0,0.0,0.0,-0.000175,0.01,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.000893,-0.0625,-0.04,0.0,0.000282,0.0,0.0,0.0,0.000003,0.0,0.0,0.0,0.01
960,1.698840e+09,12182,20,45934749,20.045935,vehicle_link,0.0,0.0,0.0,1.0,-1.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.000175,0.01,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.000893,-0.0625,-0.04,0.0,0.000282,0.0,0.0,0.0,0.000003,0.0,0.0,0.0,0.01
961,1.698840e+09,12183,20,65897553,20.065898,vehicle_link,0.0,0.0,0.0,1.0,-1.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.000524,0.01,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.000893,-0.0625,-0.04,0.0,0.000282,0.0,0.0,0.0,0.000003,0.0,0.0,0.0,0.01


In [6]:
# 计算新列
df_map_path['secs.nsecs'] = df_map_path['header.stamp.secs'] + df_map_path['header.stamp.nsecs'] * 1e-9
df_map_path['secs.nsecs'] = df_map_path['secs.nsecs'].round(9)
# 调整列的位置，将 'secs.nsecs' 放到 'header.stamp.nsecs' 后面
columns = df_map_path.columns.tolist()
index = columns.index('header.stamp.nsecs') + 1  # 获取目标列的索引并加1
columns.insert(index, columns.pop(columns.index('secs.nsecs')))  # 移动 secs.nsecs 列

# 重新排列列顺序
df_map_path = df_map_path[columns]

In [133]:
# z, cov_z = func_z(k, dt_step)   
import ast
from sympy import symbols, Matrix

# 定义函数 func_z
def func_z(k, dt_step):
    t = k * dt_step

    # 从 df_map_path 中找到与 t 值最接近的 index
    idx                    = (df_map_path['secs.nsecs'] - t).abs().idxmin()
    x                      = df_map_path.loc[idx, 'pose.pose.position.x']
    y                      = df_map_path.loc[idx, 'pose.pose.position.y']
    theta                  = df_map_path.loc[idx, 'pose.pose.orientation.z']
    z                      = Matrix([x, y, theta])
    
    cov_pose_str           = df_map_path.loc[idx, 'pose.covariance']
    cov_pose_parsed_matrix = ast.literal_eval(cov_pose_str)
    cov_x                  = cov_pose_parsed_matrix[0]
    cov_y                  = cov_pose_parsed_matrix[7]
    cov_xy                 = cov_pose_parsed_matrix[1]
    cov_theta              = cov_pose_parsed_matrix[14]
    cov_z                  = np.array([cov_x, cov_y, cov_xy, cov_theta])
    
    # cov_pose_str           = df_map_path.loc[idx, 'pose.covariance']
    # cov_pose_parsed_matrix = ast.literal_eval(cov_pose_str)
    # cov_pose_matrix        = np.array(cov_pose_parsed_matrix).reshape(6, 6)
    # cov_x                  = cov_pose_matrix[0, 0]
    # cov_y                  = cov_pose_matrix[1, 1]
    # cov_xy                 = cov_pose_matrix[1, 0]
    # cov_theta              = cov_pose_matrix[2, 2]
    # cov_z                  = np.array([cov_x, cov_y, cov_xy, cov_theta])

    return z, cov_z, idx
    
k = 10000
dt_step = 0.1
z, cov_z, idx = func_z(k, dt_step)   

In [134]:
z

Matrix([
[  -719.413248181255],
[    902.60567716048],
[-0.0460239415539503]])

In [135]:
cov_z

array([0.007521, 0.034178, 0.002606, 0.      ])

In [143]:
cov_x = cov_z[0]
cov_x

np.float64(0.007521273002702677)

In [144]:
cov_x*10

np.float64(0.07521273002702678)

In [139]:
idx

99586

In [140]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_colwidth', None)
# df_map_path.iloc[86417]
df_map_path.iloc[idx]

Time                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 1698840637.006871
header.seq                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [137]:
cov_pose_str = df_map_path.loc[idx, 'pose.covariance']
cov_pose_parsed_matrix = ast.literal_eval(cov_pose_str)
cov_pose_matrix        = np.array(cov_pose_parsed_matrix).reshape(6, 6)
cov_pose_matrix

array([[ 0.007521,  0.002606,  0.      ,  0.      ,  0.      ,  0.000235],
       [ 0.002606,  0.034178,  0.      ,  0.      ,  0.      ,  0.002477],
       [ 0.      ,  0.      ,  0.      ,  0.      , -0.      ,  0.      ],
       [ 0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ],
       [ 0.      ,  0.      , -0.      ,  0.      ,  0.      ,  0.      ],
       [ 0.000235,  0.002477,  0.      ,  0.      ,  0.      ,  0.000689]])

In [138]:
cov_pose_parsed_matrix[0]

0.007521273002702677

In [63]:
# u, cov_v, cov_omega = func_u(k, dt_step)
import ast
# 定义函数 func_u
def func_u(k, dt_step):
    t = k * dt_step

    # 从 df_map_path 中找到与 t 值最接近的 index
    idx_v               = (df_map_path['secs.nsecs'] - t).abs().idxmin()
    v                   = df_map_path.loc[idx_v, 'twist.twist.linear.x']
    cov_v_str           = df_map_path.loc[idx_v, 'twist.covariance']
    cov_v_parsed_matrix = ast.literal_eval(cov_v_str)
    cov_v_matrix        = np.array(cov_v_parsed_matrix ).reshape(6, 6)
    cov_v               = cov_v_matrix[0, 0]
    
    # 从 df_imu 中找到与 t 值最接近的 index
    idx_omega = (df_imu['secs.nsecs'] - t).abs().idxmin()
    omega     = df_imu.loc[idx_omega, 'angular_velocity.z']
    cov_omega = df_imu.loc[idx_omega, 'angular_velocity_covariance_8']
    
    u = np.array([v, omega])
    cov_u = np.array([cov_v, cov_omega])
    return u, cov_u
    
k = 20
dt_step = 0.1
u, cov_u = func_u(k, dt_step)

In [64]:
u[0]

np.float64(-0.0087621249638975)

In [56]:
print(u)
print(cov_u)

[-0.008762 -0.000175]
[0.003681 0.000893]


In [31]:
cov_v = cov_u[0]
cov_v

np.float64(0.0030086447490765907)

In [20]:
parsed_matrix

(0.0030086447490765907,
 1.5561659331312726e-12,
 0.0,
 0.0,
 0.0,
 9.675901077600093e-16,
 1.5561659331321106e-12,
 0.0089520310042253,
 0.0,
 0.0,
 0.0,
 -1.506543065392379e-16,
 0.0,
 0.0,
 9.927269263761278e-07,
 -9.88369402700485e-27,
 4.562309756717653e-26,
 0.0,
 0.0,
 0.0,
 -9.883694027004851e-27,
 9.975578884450774e-07,
 5.201126031577243e-35,
 0.0,
 0.0,
 0.0,
 4.5623097567176534e-26,
 5.20112603158178e-35,
 9.975578884450774e-07,
 0.0,
 9.675901077600107e-16,
 -1.50654306539238e-16,
 0.0,
 0.0,
 0.0,
 2.6757690447115207e-05)

In [39]:
import numpy as np
import pandas as pd

# 假设 df_gps_path 是原始 DataFrame
pose_covariance_column = df_map_path['pose.covariance']

# 定义转换函数，将字符串协方差矩阵转成数值矩阵
def convert_to_matrix(covariance_str):
    """
    将协方差矩阵的字符串表示转换为数值矩阵
    :param covariance_str: 协方差的字符串形式，如 "(1.0, 0.0, ...)"
    :return: 6x6 数值矩阵
    """
    # 去掉括号并拆分为列表
    pose_covariance_list = covariance_str.strip('()').split(',')
    # 转换为浮点数列表
    pose_covariance_floats = [float(value.strip()) for value in pose_covariance_list]
    # 转换为6x6矩阵
    covariance_matrix = np.array(pose_covariance_floats).reshape(6, 6)
    return covariance_matrix

# 对 pose.covariance 列的每个值应用转换函数
pose_covariance_matrices = pose_covariance_column.apply(convert_to_matrix)

# 将矩阵结果加入到新的 DataFrame 中
df_matrices = pd.DataFrame({
    "pose_covariance_matrix": pose_covariance_matrices
}, index=df_map_path.index)

# 设置打印选项
np.set_printoptions(linewidth=200, precision=6, suppress=True)

# 自定义打印函数
def print_matrices(df):
    for idx, row in df.iterrows():
        print(f"Index: {idx}")
        print(row['pose_covariance_matrix'])  # 打印完整的矩阵
        print("-" * 40)  # 分隔符

# 重新运行打印函数
print_matrices(df_matrices.iloc[3127:3128])

Index: 3127
[[ 0.001212  0.000006  0.        0.        0.       -0.      ]
 [ 0.000006  0.001242  0.        0.        0.       -0.      ]
 [ 0.        0.        0.000001 -0.        0.        0.      ]
 [ 0.        0.       -0.        0.000001  0.        0.      ]
 [ 0.        0.        0.        0.        0.000001  0.      ]
 [-0.       -0.        0.        0.        0.        0.000086]]
----------------------------------------


In [40]:
print_matrices(df_matrices.iloc[6255:6256])

Index: 6255
[[ 0.001735 -0.000012  0.        0.        0.       -0.000008]
 [-0.000012  0.002025  0.        0.        0.        0.000072]
 [ 0.        0.        0.000001  0.       -0.        0.      ]
 [ 0.        0.        0.        0.000001  0.        0.      ]
 [ 0.        0.       -0.        0.        0.000001  0.      ]
 [-0.000008  0.000072  0.        0.        0.        0.00011 ]]
----------------------------------------


In [13]:
# 假设 df_gps_path 是原始 DataFrame
pose_covariance_column = df_map_path['twist.covariance']

# 定义转换函数，将字符串协方差矩阵转成数值矩阵
def convert_to_matrix(covariance_str):
    """
    将协方差矩阵的字符串表示转换为数值矩阵
    :param covariance_str: 协方差的字符串形式，如 "(1.0, 0.0, ...)"
    :return: 6x6 数值矩阵
    """
    # 去掉括号并拆分为列表
    pose_covariance_list = covariance_str.strip('()').split(',')
    # 转换为浮点数列表
    pose_covariance_floats = [float(value.strip()) for value in pose_covariance_list]
    # 转换为6x6矩阵
    covariance_matrix = np.array(pose_covariance_floats).reshape(6, 6)
    return covariance_matrix

# 对 pose.covariance 列的每个值应用转换函数
pose_covariance_matrices = pose_covariance_column.apply(convert_to_matrix)

In [14]:
pose_covariance_matrices.iloc[1944]

array([[ 3.00864475e-03,  1.55616593e-12,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  9.67590108e-16],
       [ 1.55616593e-12,  8.95203100e-03,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00, -1.50654307e-16],
       [ 0.00000000e+00,  0.00000000e+00,  9.92726926e-07,
        -9.88369403e-27,  4.56230976e-26,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00, -9.88369403e-27,
         9.97557888e-07,  5.20112603e-35,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  4.56230976e-26,
         5.20112603e-35,  9.97557888e-07,  0.00000000e+00],
       [ 9.67590108e-16, -1.50654307e-16,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  2.67576904e-05]])

In [ ]:
pose_covariance_matrices.iloc[1944][0,0]

In [ ]:
import numpy as np

# 设置打印选项
np.set_printoptions(linewidth=200, precision=6, suppress=True)

# 自定义打印函数
def print_matrices(df):
    for idx, row in df.iterrows():
        print(f"Index: {idx}")
        print(row['pose_covariance_matrix'])  # 打印完整的矩阵
        print("-" * 40)  # 分隔符

# 重新运行打印函数
print_matrices(df_matrices.sample(n=10))
